In [1]:
import Pkg
Pkg.add("DifferentialEquations")

    Updating registry at `C:\Users\Miguel\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Miguel\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Miguel\.julia\environments\v1.9\Manifest.toml`


## Dividing into 2 segments

In [7]:
using LinearAlgebra
function segmented_trajectory(c_air::Float64, v_wind::Array{Float64,1},
        SourcePoint::Array{Float64,1}, ReceiverPoint::Array{Float64,1},
        SegmentsCoords::Array{Float64,2})
    
    num_segments = size(SegmentsCoords, 1)
    
    # Initialize the total travel time
    total_travel_time = 0.0

    # Iterate over the segments (including the extra segment)
    for i in 1:num_segments+1
        if i == 1
            # First segment: SourcePoint to SegmentsCoords[1, :]
            start_point = SourcePoint
            end_point = SegmentsCoords[i, :]
        elseif i == num_segments+1
            # Last segment: SegmentsCoords[num_segments, :] to ReceiverPoint
            start_point = SegmentsCoords[num_segments, :]
            end_point = ReceiverPoint
        else
            # Intermediate segments
            start_point = SegmentsCoords[i-1, :]
            end_point = SegmentsCoords[i, :]
        end
        
        vector_i = end_point - start_point
        n_i = vector_i / norm(vector_i)
        
        c_1 = c_air + dot(v_wind, n_i)
        c_2 = c_air + dot(v_wind, n_i)
        
        # Add travel time of the current segment to the total time
        total_travel_time += norm(vector_i) * (2 / (c_1+c_2))
    end
    return total_travel_time
end
using Optim
#Arguments
c_air=340.;
v_wind=[0.,0.,0.];
SourcePoint=[0.,0.,0.];
ReceiverPoint=[1000.,1000.,1000.];
#Optimize the coordinates of the union of segments
function objective(SegmentsCoords)
    traveltime = segmented_trajectory(c_air, v_wind, SourcePoint, ReceiverPoint, SegmentsCoords)
    return traveltime
end

num_segments=2;
initial_SegmentsCoords = zeros(num_segments-1, 3); 
for i in 1:num_segments-1
    alpha = i / num_segments
    initial_SegmentsCoords[i, :] = SourcePoint + alpha * (ReceiverPoint - SourcePoint) #Recta troceada 
    #desde la fuente al receptor como condición inicial
end
result = optimize(objective, initial_SegmentsCoords);
optimal_SegmentsCoords = result.minimizer;
total_time = segmented_trajectory(c_air, v_wind, SourcePoint, ReceiverPoint, optimal_SegmentsCoords)
println("The total travel time for the segmented trajectory is: ", total_time, " seconds")
println(optimal_SegmentsCoords)

The total travel time for the segmented trajectory is: 5.094267081084933 seconds
[500.0 500.0 500.0]



## Height-dependent speed of sound ( c(z) ).

In [8]:
function c_air_TZ(T_z0,z)
    c_air=331*sqrt((T_z0+273.15-z/300)/273.15) #Introduzco T_Z0 en grados centígrados por comodidad
    return c_air
end

c_air_TZ (generic function with 1 method)

In [9]:
using LinearAlgebra
function segmented_trajectory_TZ(c_air_TZ, T,  v_wind::Array{Float64, 1},
        SourcePoint::Array{Float64, 1}, ReceiverPoint::Array{Float64, 1},
        SegmentsCoords::Array{Float64, 2})
    
    # Check if all elements in SegmentsCoords are positive
    if any(SegmentsCoords[:,3] .< 0)
        throw(ArgumentError("SegmentsCoords must contain only positive floats"))
    end

    num_segments = size(SegmentsCoords, 1)
    # Initialize the total travel time
    total_travel_time = 0.0
    
    # Iterate over the segments (including the extra segment)
    for i in 1:num_segments+1
        if i == 1
            # First segment: SourcePoint to SegmentsCoords[1, :]
            start_point = SourcePoint
            end_point = SegmentsCoords[i, :]
        elseif i == num_segments+1
            # Last segment: SegmentsCoords[num_segments, :] to ReceiverPoint
            start_point = SegmentsCoords[num_segments, :]
            end_point = ReceiverPoint
        else
            # Intermediate segments
            start_point = SegmentsCoords[i-1, :]
            end_point = SegmentsCoords[i, :]
        end
        
        # Calculate characteristics of the current segment
        vector_i = end_point - start_point
        n_i = vector_i / norm(vector_i)
        
        c_1 = c_air_TZ(T, start_point[3]) + dot(v_wind, n_i)
        c_2 = c_air_TZ(T, end_point[3]) + dot(v_wind, n_i)
        
        # Add travel time of the current segment to the total time
        total_travel_time += norm(vector_i) * (2 / (c_1+c_2))
    end
    return total_travel_time
end
using Optim
#Arguments
T=20.;
v_wind=[0.,0.,0.];
SourcePoint=[0.,0.,0.];
ReceiverPoint=[1000.,1000.,1000.];
#Optimize the coordinates of the union of segments
function objective(SegmentsCoords)
    traveltime = segmented_trajectory_TZ(c_air_TZ, T , v_wind, SourcePoint, ReceiverPoint, SegmentsCoords)
    return traveltime
end

num_segments=2;
initial_SegmentsCoords = zeros(num_segments-1, 3); 
for i in 1:num_segments-1
    alpha = i / num_segments
    initial_SegmentsCoords[i, :] = SourcePoint + alpha * (ReceiverPoint - SourcePoint) #Recta troceada 
    #desde la fuente al receptor como condición inicial
end
result = optimize(objective, initial_SegmentsCoords);
optimal_SegmentsCoords = result.minimizer;
total_time = segmented_trajectory_TZ(c_air_TZ, T, v_wind, SourcePoint, ReceiverPoint, optimal_SegmentsCoords)
println("The total travel time for the segmented trajectory is: ", total_time, " seconds")
println(optimal_SegmentsCoords)

The total travel time for the segmented trajectory is: 5.065557660469514 seconds
[531.7802114482043 531.8151321821044 529.6639876008126]


In [10]:
function c_air_TZ(T_z0,z)
    c_air=331*sqrt((T_z0+273.15-z/300)/273.15) #Introduzco T_Z0 en grados centígrados por comodidad
    return c_air
end
using LinearAlgebra
function segmented_trajectory_TZ(c_air_TZ, T,  v_wind::Array{Float64, 1},
        SourcePoint::Array{Float64, 1}, ReceiverPoint::Array{Float64, 1},
        SegmentsCoords::Array{Float64, 2})
    
    # Check if all elements in SegmentsCoords are positive
    if any(SegmentsCoords[:,3] .< 0)
        throw(ArgumentError("SegmentsCoords must contain only positive floats"))
    end

    num_segments = size(SegmentsCoords, 1)
    # Initialize the total travel time
    total_travel_time = 0.0
    
    # Iterate over the segments (including the extra segment)
    for i in 1:num_segments+1
        if i == 1
            # First segment: SourcePoint to SegmentsCoords[1, :]
            start_point = SourcePoint
            end_point = SegmentsCoords[i, :]
        elseif i == num_segments+1
            # Last segment: SegmentsCoords[num_segments, :] to ReceiverPoint
            start_point = SegmentsCoords[num_segments, :]
            end_point = ReceiverPoint
        else
            # Intermediate segments
            start_point = SegmentsCoords[i-1, :]
            end_point = SegmentsCoords[i, :]
        end
        
        # Calculate characteristics of the current segment
        vector_i = end_point - start_point
        n_i = vector_i / norm(vector_i)
        
        c_1 = c_air_TZ(T, start_point[3]) + dot(v_wind, n_i)
        c_2 = c_air_TZ(T, end_point[3]) + dot(v_wind, n_i)
        
        # Add travel time of the current segment to the total time
        total_travel_time += norm(vector_i) * (2 / (c_1+c_2))
    end
    return total_travel_time
end
using Optim
#Arguments
T=20.;
v_wind=[0.,0.,0.];
SourcePoint=[0.,0.,0.];
ReceiverPoint=[1000.,1000.,1000.];
#Optimize the coordinates of the union of segments
function objective(SegmentsCoords)
    traveltime = segmented_trajectory_TZ(c_air_TZ, T , v_wind, SourcePoint, ReceiverPoint, SegmentsCoords)
    return traveltime
end

num_segments=10;
initial_SegmentsCoords = zeros(num_segments-1, 3); 
for i in 1:num_segments-1
    alpha = i / num_segments
    initial_SegmentsCoords[i, :] = SourcePoint + alpha * (ReceiverPoint - SourcePoint) #Recta troceada 
    #desde la fuente al receptor como condición inicial
end
result = optimize(objective, initial_SegmentsCoords);
optimal_SegmentsCoords = result.minimizer;
total_time = segmented_trajectory_TZ(c_air_TZ, T, v_wind, SourcePoint, ReceiverPoint, optimal_SegmentsCoords)
println("The total travel time for the segmented trajectory is: ", total_time, " seconds")
println(optimal_SegmentsCoords)

The total travel time for the segmented trajectory is: 5.065554330463617 seconds
[107.36167955374614 107.39593533443515 106.55814164194257; 210.418024160734 210.47854431301943 209.03066992368247; 312.37603632594335 312.45466190991095 310.57795168703063; 413.0915408392108 413.1893743470757 411.06238857189794; 513.7391812709276 513.8408828655319 511.64953788526276; 613.156653326566 613.2520064799917 611.1689701932883; 713.2476161867609 713.3218526296733 711.5370694382818; 813.5566152508934 813.6095630316939 812.2862516929229; 913.2709345896372 913.29826159589 912.6060446949131]


In [11]:
using Plots 

#Tomo para cada z , ecuación de la recta y-x=0
F_optimal_SegmentsCoords=vcat(reshape(SourcePoint, 1, :), optimal_SegmentsCoords, reshape(ReceiverPoint, 1, :))
z=F_optimal_SegmentsCoords[:, 3]
aux=sqrt.((F_optimal_SegmentsCoords[:, 3]-F_optimal_SegmentsCoords[:, 1]).^2+(F_optimal_SegmentsCoords[:, 2]-F_optimal_SegmentsCoords[:, 1]).^2)
plot!(z,aux,label=false, markershape=:diamond)
xlabel!("z (m)")
ylabel!("Deviation to the straight line (m)")
#savefig("SegmentedTrajectory.png")

"C:\\Users\\Miguel\\Desktop\\TFM\\Codigos Julia\\SegmentedTrajectory.png"

## Analyze what is the maximum number of segments for which the optimizer returns something different from the initial condition.

In [12]:
using Optim
#Arguments
T=20.;
v_wind=[0.,0.,0.];
SourcePoint=[0.,0.,0.];
ReceiverPoint=[1000.,1000.,1000.];
#Optimize the coordinates of the union of segments
function objective(SegmentsCoords)
    traveltime = segmented_trajectory_TZ(c_air_TZ, T , v_wind, SourcePoint, ReceiverPoint, SegmentsCoords)
    return traveltime
end

for i in 2:50
    num_segments=i;
    initial_SegmentsCoords = zeros(num_segments-1, 3); 
    for j in 1:num_segments-1
        alpha = j / num_segments
        initial_SegmentsCoords[j, :] = SourcePoint + alpha * (ReceiverPoint - SourcePoint) #Recta troceada 
        #desde la fuente al receptor como condición inicial
    end
    result = optimize(objective, initial_SegmentsCoords);
    optimal_SegmentsCoords = result.minimizer;
    total_time = segmented_trajectory_TZ(c_air_TZ, T, v_wind, SourcePoint, ReceiverPoint, optimal_SegmentsCoords)
    if optimal_SegmentsCoords == initial_SegmentsCoords
        num_segments_stop = num_segments-1
        println(num_segments_stop, " is the maximum number of segments for which Optim optimizes the initial condition")
        break
    end
    println("Num segments: ", num_segments, ". Traveltime:", total_time, " s")
end

Num segments: 2. Traveltime:5.065557660469514 s
Num segments: 3. Traveltime:5.065555710183466 s
Num segments: 4. Traveltime:5.065555039856497 s
Num segments: 5. Traveltime:5.065554730467388 s
Num segments: 6. Traveltime:5.065554556623453 s
Num segments: 7. Traveltime:5.065554454030572 s
Num segments: 8. Traveltime:5.065554387564779 s
Num segments: 9. Traveltime:5.065554342630008 s
Num segments: 10. Traveltime:5.065554330463617 s
Num segments: 11. Traveltime:5.0655544182695795 s
Num segments: 12. Traveltime:5.06555580536549 s
Num segments: 13. Traveltime:5.065558548261519 s
Num segments: 14. Traveltime:5.065561740738669 s
Num segments: 15. Traveltime:5.065562411579639 s
Num segments: 16. Traveltime:5.065565547537591 s
Num segments: 17. Traveltime:5.065565982369442 s
Num segments: 18. Traveltime:5.065566594254764 s
Num segments: 19. Traveltime:5.065567008658724 s
Num segments: 20. Traveltime:5.065567665831462 s
Num segments: 21. Traveltime:5.065567830559983 s
Num segments: 22. Traveltime

## Iterate initial condition.

In [13]:
using Optim
#Arguments
T=20.;
v_wind=[0.,0.,0.];
SourcePoint=[0.,0.,0.];
ReceiverPoint=[1000.,1000.,1000.];
#Optimize the coordinates of the union of segments
function objective(SegmentsCoords)
    traveltime = segmented_trajectory_TZ(c_air_TZ, T , v_wind, SourcePoint, ReceiverPoint, SegmentsCoords)
    return traveltime
end

initial_SegmentsCoords = zeros(1, 3); #c.i.
for i in 1:10
    if i==1
        initial_SegmentsCoords[1, :] = SourcePoint +  (ReceiverPoint - SourcePoint)/2
    end
    result = optimize(objective, initial_SegmentsCoords);
    optimal_SegmentsCoords = result.minimizer;
    total_time = segmented_trajectory_TZ(c_air_TZ, T, v_wind, SourcePoint, ReceiverPoint, optimal_SegmentsCoords)
    println("Num segments: ", Int(size(optimal_SegmentsCoords,1)+1), ". Traveltime:", total_time, " s")
    
    copy_optimal_SegmentsCoords=zeros(2*size(optimal_SegmentsCoords,1)+1,3)
    for j in 1:2*size(optimal_SegmentsCoords,1)+1
        if j==1
            copy_optimal_SegmentsCoords[j,:]=(SourcePoint+optimal_SegmentsCoords[1,:])/2
        elseif j==2*size(optimal_SegmentsCoords,1)+1
            copy_optimal_SegmentsCoords[j,:]=(optimal_SegmentsCoords[end,:]+ReceiverPoint)/2
        else
            if iseven(j)
                copy_optimal_SegmentsCoords[j,:]=optimal_SegmentsCoords[Int(j/2),:]
            else
                copy_optimal_SegmentsCoords[j,:]=(optimal_SegmentsCoords[Int((j-1)/2),:]+optimal_SegmentsCoords[Int((j+1)/2),:])/2
            end
        end
    end
    initial_SegmentsCoords = copy(copy_optimal_SegmentsCoords)
end

Num segments: 2. Traveltime:5.065557660469514 s
Num segments: 4. Traveltime:5.065555053550677 s
Num segments: 8. Traveltime:5.065554398734178 s
Num segments: 16. Traveltime:5.065554313395924 s
Num segments: 32. Traveltime:5.065554313395923 s
Num segments: 64. Traveltime:5.065554313395924 s
Num segments: 128. Traveltime:5.065554313395924 s
Num segments: 256. Traveltime:5.065554313395925 s
Num segments: 512. Traveltime:5.065554313395925 s
Num segments: 1024. Traveltime:5.065554313395918 s
